In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append('../')

from typing import List,Dict
from elmo_on_md.data_loaders.sentiment_loader import SentimentLoader
from elmo_on_md.evaluation.sentiment_analysis import SentimentAnalysis
from elmo_on_md.evaluation.model_loader import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
loader = SentimentLoader()
sentiment_data = loader.load_data()

sentences = sentiment_data['train']['sentences']
labels = sentiment_data['train']['labels']

tokens_train,tokens_test, labels_train,labels_test = train_test_split(sentences, labels, test_size=0.2, random_state=1)
train_set = {'sentences':tokens_train,'labels':labels_train}
validate_set = {'sentences':tokens_test,'labels':labels_test}

In [3]:
elmo = load_model('original', batch_size = 32)
sentiment = SentimentAnalysis(elmo,lr=1e-4)

sentiment.train(train_set,validate_set,n_epochs=200, batch_size = 64, tb_dir='baseline')

2019-08-30 11:33:54,842 INFO: char embedding size: 2289
2019-08-30 11:33:55,544 INFO: word embedding size: 189561
2019-08-30 11:34:00,310 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(189561, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2289, 50, padding_idx=2286)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_fe

Epoch: 0	 Train Loss: 1.251713752746582	 Validation Loss: 0.9798790216445923
Epoch: 1	 Train Loss: 1.238234043121338	 Validation Loss: 0.8694154620170593
Epoch: 2	 Train Loss: 1.1014785766601562	 Validation Loss: 0.8371168375015259
Epoch: 3	 Train Loss: 0.7940273880958557	 Validation Loss: 0.8148648738861084
Epoch: 4	 Train Loss: 0.627234697341919	 Validation Loss: 0.8098123669624329
Epoch: 5	 Train Loss: 0.5830479860305786	 Validation Loss: 0.8079448342323303
Epoch: 6	 Train Loss: 0.6025349497795105	 Validation Loss: 0.820486843585968
Epoch: 7	 Train Loss: 0.5655662417411804	 Validation Loss: 0.809010922908783
Epoch: 8	 Train Loss: 0.5696036219596863	 Validation Loss: 0.7868145704269409
Epoch: 9	 Train Loss: 0.5614644885063171	 Validation Loss: 0.7845606207847595
Epoch: 10	 Train Loss: 0.5696258544921875	 Validation Loss: 0.8034985065460205
Epoch: 11	 Train Loss: 0.5669284462928772	 Validation Loss: 0.8192839622497559
Epoch: 12	 Train Loss: 0.5548382997512817	 Validation Loss: 0.81304

Epoch: 104	 Train Loss: 0.5514719486236572	 Validation Loss: 0.7779171466827393
Epoch: 105	 Train Loss: 0.5515329837799072	 Validation Loss: 0.7973482012748718
Epoch: 106	 Train Loss: 0.551449716091156	 Validation Loss: 0.7974193692207336
Epoch: 107	 Train Loss: 0.5514612793922424	 Validation Loss: 0.7956085205078125
Epoch: 108	 Train Loss: 0.5514475107192993	 Validation Loss: 0.8092987537384033
Epoch: 109	 Train Loss: 0.551445484161377	 Validation Loss: 0.8000390529632568
Epoch: 110	 Train Loss: 0.5514453053474426	 Validation Loss: 0.8003888130187988
Epoch: 111	 Train Loss: 0.5514583587646484	 Validation Loss: 0.7809305787086487
Epoch: 112	 Train Loss: 0.5514459609985352	 Validation Loss: 0.7929240465164185
Epoch: 113	 Train Loss: 0.5514534115791321	 Validation Loss: 0.8291666507720947
Epoch: 114	 Train Loss: 0.5514469742774963	 Validation Loss: 0.8105924725532532
Epoch: 115	 Train Loss: 0.5515406131744385	 Validation Loss: 0.7989011406898499
Epoch: 116	 Train Loss: 0.5514614582061768

#### results

In [4]:
# train_preds = sentiment.predict(sentiment_data['train'])
# print(confusion_matrix(sentiment_data['train']['labels'],train_preds))
y_pred = sentiment.predict(sentiment_data['test'])
y_true = sentiment_data['test']['labels']

print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true, y_pred, target_names=['positive','negative','neutral']))

2019-08-30 11:41:09,505 INFO: 80 batches, avg len: 17.1
2019-08-30 11:41:13,236 INFO: Finished 1000 sentences.
2019-08-30 11:41:16,681 INFO: Finished 2000 sentences.


[[1505  178   15]
 [ 105  669   16]
 [  17   23   32]]
              precision    recall  f1-score   support

    positive       0.93      0.89      0.91      1698
    negative       0.77      0.85      0.81       790
     neutral       0.51      0.44      0.47        72

    accuracy                           0.86      2560
   macro avg       0.73      0.73      0.73      2560
weighted avg       0.87      0.86      0.86      2560



# Our Model

In [3]:
elmo = load_model('pos_weight8_lr-4_new_tags_30epochs', batch_size = 32)
sentiment = SentimentAnalysis(elmo,lr=1e-4)

sentiment.train(train_set,validate_set,n_epochs=200, batch_size = 64, tb_dir='new_elmo')

2019-08-30 11:42:14,912 INFO: 129 batches, avg len: 16.9
2019-08-30 11:42:18,289 INFO: Finished 1000 sentences.
2019-08-30 11:42:20,587 INFO: Finished 2000 sentences.
2019-08-30 11:42:22,430 INFO: Finished 3000 sentences.
2019-08-30 11:42:24,966 INFO: Finished 4000 sentences.
2019-08-30 11:42:27,043 INFO: Finished 5000 sentences.
2019-08-30 11:42:29,027 INFO: Finished 6000 sentences.
2019-08-30 11:42:31,315 INFO: Finished 7000 sentences.
2019-08-30 11:42:34,019 INFO: Finished 8000 sentences.
2019-08-30 11:42:38,765 INFO: 33 batches, avg len: 17.2
2019-08-30 11:42:41,449 INFO: Finished 1000 sentences.
2019-08-30 11:42:43,643 INFO: Finished 2000 sentences.
..\elmo_on_md\evaluation\sentiment_analysis.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)


Epoch: 0	 Train Loss: 1.278416633605957	 Validation Loss: 0.9915648102760315
Epoch: 1	 Train Loss: 1.2893081903457642	 Validation Loss: 0.8957425951957703
Epoch: 2	 Train Loss: 1.154569387435913	 Validation Loss: 0.8686726689338684
Epoch: 3	 Train Loss: 0.8989688158035278	 Validation Loss: 0.8545435667037964
Epoch: 4	 Train Loss: 0.60067218542099	 Validation Loss: 0.8538553714752197
Epoch: 5	 Train Loss: 0.6054528951644897	 Validation Loss: 0.8255987763404846
Epoch: 6	 Train Loss: 0.5649961829185486	 Validation Loss: 0.8317887187004089
Epoch: 7	 Train Loss: 0.5590201020240784	 Validation Loss: 0.8651028871536255
Epoch: 8	 Train Loss: 0.565280556678772	 Validation Loss: 0.8863596320152283
Epoch: 9	 Train Loss: 0.5617823004722595	 Validation Loss: 0.873504638671875
Epoch: 10	 Train Loss: 0.566034734249115	 Validation Loss: 0.8939685225486755
Epoch: 11	 Train Loss: 0.5574159026145935	 Validation Loss: 0.8642645478248596
Epoch: 12	 Train Loss: 0.5571596622467041	 Validation Loss: 0.8806497

Epoch: 104	 Train Loss: 0.5514453053474426	 Validation Loss: 0.8426187038421631
Epoch: 105	 Train Loss: 0.5514470934867859	 Validation Loss: 0.8537213802337646
Epoch: 106	 Train Loss: 0.5514490604400635	 Validation Loss: 0.8516772389411926
Epoch: 107	 Train Loss: 0.5514452457427979	 Validation Loss: 0.8479706048965454
Epoch: 108	 Train Loss: 0.5514592528343201	 Validation Loss: 0.8502740859985352
Epoch: 109	 Train Loss: 0.5515186786651611	 Validation Loss: 0.8617008328437805
Epoch: 110	 Train Loss: 0.5514453053474426	 Validation Loss: 0.8458512425422668
Epoch: 111	 Train Loss: 0.551447868347168	 Validation Loss: 0.8552643060684204
Epoch: 112	 Train Loss: 0.5515409708023071	 Validation Loss: 0.8007053732872009
Epoch: 113	 Train Loss: 0.5514780879020691	 Validation Loss: 0.8411164283752441
Epoch: 114	 Train Loss: 0.5514653921127319	 Validation Loss: 0.8619568943977356
Epoch: 115	 Train Loss: 0.5514633655548096	 Validation Loss: 0.8406601548194885
Epoch: 116	 Train Loss: 0.551455080509185

#### results

In [4]:
y_pred = sentiment.predict(sentiment_data['test'])
y_true = sentiment_data['test']['labels']

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=['positive','negative','neutral']))

2019-08-30 11:48:44,717 INFO: 40 batches, avg len: 17.1
2019-08-30 11:48:46,674 INFO: Finished 1000 sentences.
2019-08-30 11:48:49,457 INFO: Finished 2000 sentences.


[[1506  175   17]
 [ 142  626   22]
 [  23   18   31]]
              precision    recall  f1-score   support

    positive       0.90      0.89      0.89      1698
    negative       0.76      0.79      0.78       790
     neutral       0.44      0.43      0.44        72

    accuracy                           0.84      2560
   macro avg       0.70      0.70      0.70      2560
weighted avg       0.85      0.84      0.85      2560

